# Q2. Graph Analysis

## Install PySpark

In [1]:
# Install latest version of spark. If error, check the latest and replace "spark-2.4.4"
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://www-us.apache.org/dist/spark/spark-2.4.4/spark-2.4.4-bin-hadoop2.7.tgz
!tar xf spark-2.4.4-bin-hadoop2.7.tgz
!pip install -q findspark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.4-bin-hadoop2.7"
import findspark
findspark.init()

/bin/sh: apt-get: command not found
/bin/sh: wget: command not found
tar: Error opening archive: Failed to open 'spark-2.4.4-bin-hadoop2.7.tgz'


IndexError: list index out of range

In [2]:
import numpy as np
import pandas as pd

from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
from pyspark import SparkConf, SparkContext
import pyspark
import sys
from collections import defaultdict

In [0]:
# Configure Spark
conf = SparkConf()
sc = SparkContext.getOrCreate(conf=conf)
# The directory for the file
filename = "/content/gdrive/My Drive/BigData/q1.txt"

In [0]:
# Finished. Return RDD
def getData(sc, filename):
    """
    Load data from raw text file into RDD and transform.
    Hint: transfromation you will use: map(<lambda function>).
    Args:
        sc (SparkContext): spark context.
        filename (string): hw2.txt cloud storage URI.
    Returns:
        RDD: RDD list of tuple of (<User>, [friend1, friend2, ... ]),
        each user and a list of user's friends
    """
    # read text file into RDD
    data = sc.textFile(filename)

    # TODO: implement your logic here
    data = data.map(lambda line: np.array([str(x) for x in line.replace('\n','').split('\t')]))
    data = data.map(lambda p:(int(p[0]), p[1].split(',')))

    return data

In [0]:
def getEdges(line):
    # similar to mapFriends() in Q1, edges are direct friendship
    friends = line[1]
    user = line[0]

    if friends != ['']:
        for i in range(len(friends)):
            # Direct friend
            yield((int(user), int(friends[i])))
            #yield((friends[i], user))

### 1. Format data into edges and vertices

In [0]:
# Get data in proper format
data = getData(sc, filename)

In [8]:
vertices = data.map(lambda x: (x[0],))
vertices.take(5)

[(0,), (1,), (2,), (3,), (4,)]

In [9]:
edges = data.flatMap(getEdges)
edges.take(5)

[(0, 1), (0, 2), (0, 3), (0, 4), (0, 5)]

### 2. Convert the RDD to DataFrame

In [0]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local[*]") \
    .appName("Learning_Spark") \
    .getOrCreate()

In [11]:
v = spark.createDataFrame(vertices,["id"])
v.show(5)

+---+
| id|
+---+
|  0|
|  1|
|  2|
|  3|
|  4|
+---+
only showing top 5 rows



In [12]:
e = spark.createDataFrame(edges, ["user","friend"])
e.show(5)

+----+------+
|user|friend|
+----+------+
|   0|     1|
|   0|     2|
|   0|     3|
|   0|     4|
|   0|     5|
+----+------+
only showing top 5 rows



## Use GraphFrames

In [13]:
!pip install graphframes

In [14]:
!pip install pyspark command

In [0]:
!pyspark --packages graphframes：graphframes：0.5.0-spark2.1-s_2.11

In [0]:
from graphframes import *

In [0]:
g = GraphFrame(v,e)

In [71]:
from graphframes.examples import Graphs
g = Graphs(sqlContext).friends()  # Get example graph

result = g.connectedComponents()
result.select("id", "component").orderBy("component").show()

Py4JJavaError: ignored